In [16]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from pyRF.circuit import Circuit
from pyRF.feedline import TimeDomainSolution
from pyRF.Experimental.examples.feedline_time_domain import Feedline
from pyRF.Experimental.examples.quarter_wave_resonator import QuarterWave
from pyRF.feedline import TimeDomainSolution
from pyRF.Experimental.examples.coupled_resonator_feedline import ResonatorFeedline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
resonator_feedline_circuit = ResonatorFeedline('quarter_wave')
resonator_feedline_circuit.initialize()
R1 = resonator_feedline_circuit.resonator_dict['R1']
F1 = resonator_feedline_circuit.feedline_dict['F1']

In [29]:
fr = R1.get_eigenvalue()*1e8
fr

6212920435.998129

In [30]:
fr*1e-9

6.21292043599813

In [33]:
phi = R1.get_eigenfunction()
phi(0)

array(22.42575415+0.20899414j)

In [34]:
feedline_a, feedline_b = F1.get_eigenfunction(60)

In [35]:
Z0=50
phi0=1e8
clengthinv = Z0*phi0
cap = 30e-15
ja = feedline_a(10e-3)*cap*clengthinv*phi(0)
ja

(-0.00270427440119127+0.002000864273260278j)

In [ ]:
jflk = phi*c*clengthinv*psi
jresk = phi*c*clengthinv*psi

TDcoupling = jflk*jresk*np.pi**2/phi0*2

In [31]:
R1.mode_condition(fr*1e-8)

[3.879634666901867e-18, -2.081306613249581e-16]

In [25]:
F1.get_eigenfunction(60)